# Exercise on panel data

## Whose Wages do Union Raise?

Our goal here, is to redo some of the analysis from:
    
F. Vella and M. Verbeek (1998), “Whose Wages Do Unions Raise? A Dynamic Model of Unionism and Wage Rate Determination for Young Men,” Journal of Applied Econometrics 13, 163-183.

__Import the dataset wage_panel from linearmodels.datasets. Describe it and make sure
it is in a format suitable for panel data analysis. Show graphically the decrease in unionization over the period.__

__Our goal is to estimate the wage premium (if any) associated to belonging to a union.__

We first consider a model $y_{i,t} = \alpha + X_{i,t} \beta + \epsilon_{i,t}$ where $y_{i,t}$ is the wage and $X_{i,t}$ contains a vector of regressors which can explain it.

Estimate this model using OLS. What is the effect of union. What is the problem of this regression? Should we use individual or time fixed effects?

__Formulate a model with individual fixed effect. Run a panel regression. Comment the results.__

__Formulate a model with individual and time fixed effect. Run a panel regression. Comment the results.__